In [1]:
# importing the libraries
import numpy as np

# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm

# PyTorch libraries and modules
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [2]:
import pickle
def load_data(in_dir):
    f = open(in_dir,'rb')
    train_data,train_label,test_data,test_label,valid_data,valid_label,pernums_valid = pickle.load(f)
    return train_data,train_label,test_data,test_label,valid_data,valid_label,pernums_valid

data_path = 'adress.pkl'
checkpoint = 'checkpoint/'

train_data,train_label,test_data,test_label,valid_data,valid_label,pernums_valid = load_data(data_path)

In [3]:
# converting training images into torch format
train_x = train_data
train_x  = torch.from_numpy(train_x)

# converting the target into torch format
train_y = train_label
train_y = train_y.reshape(2379).astype(float);
train_y = torch.from_numpy(train_y)


# shape of training data
train_x.shape, train_y.shape

(torch.Size([2379, 300, 40, 3]), torch.Size([2379]))

In [4]:
train_y

tensor([1., 0., 0.,  ..., 1., 0., 1.], dtype=torch.float64)

In [5]:
# converting training images into torch format
val_x = valid_data
val_x  = torch.from_numpy(val_x)

# converting the target into torch format
val_y = valid_label
val_y = val_y.reshape(297).astype(float);
val_y = torch.from_numpy(val_y)
# shape of training data
val_x.shape, val_y.shape

(torch.Size([297, 300, 40, 3]), torch.Size([297]))

In [6]:
val_y

tensor([1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0.,
        1., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0.,
        1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1.,
        1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1.,
        1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1.,
        0., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1.,
        1., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1.,
        0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1.,
        1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
        1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1.,
        1., 0., 0., 1., 0., 0., 0., 0., 

In [7]:
class Net(Module):   
    def __init__(self):
        super(Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(300, 256, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(256),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(128),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=1, stride=1),
        )

        self.linear_layers = Sequential(
            Linear(128 * 20 * 1, 20)
        )

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
#         print(x)
        return x

In [8]:
# defining the model
model = Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(300, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=2560, out_features=20, bias=True)
  )
)


In [9]:
from torchsummary import summary
summary(model, (300, 40, 3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 256, 40, 3]         691,456
       BatchNorm2d-2           [-1, 256, 40, 3]             512
              ReLU-3           [-1, 256, 40, 3]               0
         MaxPool2d-4           [-1, 256, 20, 1]               0
            Conv2d-5           [-1, 128, 20, 1]         295,040
       BatchNorm2d-6           [-1, 128, 20, 1]             256
              ReLU-7           [-1, 128, 20, 1]               0
         MaxPool2d-8           [-1, 128, 20, 1]               0
            Linear-9                   [-1, 20]          51,220
Total params: 1,038,484
Trainable params: 1,038,484
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.14
Forward/backward pass size (MB): 0.82
Params size (MB): 3.96
Estimated Total Size (MB): 4.92
---------------------------------------

In [10]:
def train(epoch):
    model.train()
    tr_loss = 0
    # getting the training set
    x_train, y_train = Variable(train_x), Variable(train_y)
    # getting the validation set
    x_val, y_val = Variable(val_x), Variable(val_y)
    # converting the data into GPU format
    if torch.cuda.is_available():
        x_train = x_train.cuda()
        y_train = y_train.to(torch.int64).cuda()
        x_val = x_val.cuda()
        y_val = y_val.to(torch.int64).cuda()

    # clearing the Gradients of the model parameters
    optimizer.zero_grad()
    
    # prediction for training and validation set
    output_train = model(x_train)
    output_val = model(x_val)
    
#     print(output_train)
    # computing the training and validation loss
    with torch.autocast('cuda'):
        loss_train = criterion(output_train, y_train)
        loss_val = criterion(output_val, y_val)
    train_losses.append(loss_train)
    val_losses.append(loss_val)

    # computing the updated weights of all the model parameters
    loss_train.backward()
    optimizer.step()
    tr_loss = loss_train.item()
    if epoch%2 == 0:
        # printing the validation loss
        print('Epoch : ',epoch+1, '\t', 'loss :', loss_val)

In [11]:
# # defining the number of epochs
# n_epochs = 25
# # empty list to store training losses
# train_losses = []
# # empty list to store validation losses
# val_losses = []
# # training the model
# for epoch in range(n_epochs):
#     train(epoch)

In [12]:
torch.cuda.is_available()

True

In [13]:
# import matplotlib.pyplot as plt
# # plotting the training and validation loss
# plt.plot(train_losses, label='Training loss')
# plt.plot(val_losses, label='Validation loss')
# plt.legend()
# plt.show()

In [14]:
# # prediction for training set
# with torch.no_grad():
#     output = model(train_x.cuda())
    
# softmax = torch.exp(output).cpu()
# prob = list(softmax.numpy())
# predictions = np.argmax(prob, axis=1)

# # accuracy on training set
# accuracy_score(train_y, predictions)



In [15]:
# # prediction for validation set
# with torch.no_grad():
#     output = model(val_x.cuda())

# softmax = torch.exp(output).cpu()
# prob = list(softmax.numpy())
# predictions = np.argmax(prob, axis=1)

# # accuracy on validation set
# accuracy_score(val_y, predictions)

In [16]:
import torch
import math
import torch.nn as nn
import torch.nn.functional as F

class BasicConv(nn.Module):
    def __init__(self, in_planes, out_planes, kernel_size, stride=1, padding=0, dilation=1, groups=1, relu=True, bn=True, bias=False):
        super(BasicConv, self).__init__()
        self.out_channels = out_planes
        self.conv = nn.Conv2d(in_planes, out_planes, kernel_size=kernel_size, stride=stride, padding=padding, dilation=dilation, groups=groups, bias=bias)
        self.bn = nn.BatchNorm2d(out_planes,eps=1e-5, momentum=0.01, affine=True) if bn else None
        self.relu = nn.ReLU() if relu else None

    def forward(self, x):
        x = self.conv(x)
        if self.bn is not None:
            x = self.bn(x)
        if self.relu is not None:
            x = self.relu(x)
        return x

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class ChannelGate(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max']):
        super(ChannelGate, self).__init__()
        self.gate_channels = gate_channels
        self.mlp = nn.Sequential(
            Flatten(),
            nn.Linear(gate_channels, gate_channels // reduction_ratio),
            nn.ReLU(),
            nn.Linear(gate_channels // reduction_ratio, gate_channels)
            )
        self.pool_types = pool_types
    def forward(self, x):
        channel_att_sum = None
        for pool_type in self.pool_types:
            if pool_type=='avg':
                avg_pool = F.avg_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( avg_pool )
            elif pool_type=='max':
                max_pool = F.max_pool2d( x, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( max_pool )
            elif pool_type=='lp':
                lp_pool = F.lp_pool2d( x, 2, (x.size(2), x.size(3)), stride=(x.size(2), x.size(3)))
                channel_att_raw = self.mlp( lp_pool )
            elif pool_type=='lse':
                # LSE pool only
                lse_pool = logsumexp_2d(x)
                channel_att_raw = self.mlp( lse_pool )

            if channel_att_sum is None:
                channel_att_sum = channel_att_raw
            else:
                channel_att_sum = channel_att_sum + channel_att_raw

        scale = F.sigmoid( channel_att_sum ).unsqueeze(2).unsqueeze(3).expand_as(x)
#         print(x.shape())
        return x * scale

def logsumexp_2d(tensor):
    tensor_flatten = tensor.view(tensor.size(0), tensor.size(1), -1)
    s, _ = torch.max(tensor_flatten, dim=2, keepdim=True)
    outputs = s + (tensor_flatten - s).exp().sum(dim=2, keepdim=True).log()
    return outputs

class ChannelPool(nn.Module):
    def forward(self, x):
        return torch.cat( (torch.max(x,1)[0].unsqueeze(1), torch.mean(x,1).unsqueeze(1)), dim=1 )

class SpatialGate(nn.Module):
    def __init__(self):
        super(SpatialGate, self).__init__()
        kernel_size = 7
        self.compress = ChannelPool()
        self.spatial = BasicConv(2, 1, kernel_size, stride=1, padding=(kernel_size-1) // 2, relu=False)
    def forward(self, x):
        x_compress = self.compress(x)
        x_out = self.spatial(x_compress)
        scale = F.sigmoid(x_out) # broadcasting
        return x * scale

class CBAM(nn.Module):
    def __init__(self, gate_channels, reduction_ratio=16, pool_types=['avg', 'max'], no_spatial=False):
        super(CBAM, self).__init__()
        self.ChannelGate = ChannelGate(gate_channels, reduction_ratio, pool_types)
        self.no_spatial=no_spatial
        if not no_spatial:
            self.SpatialGate = SpatialGate()
    def forward(self, x):
        x_out = self.ChannelGate(x)
        if not self.no_spatial:
            x_out = self.SpatialGate(x_out)
        return x_out

In [17]:
class Att_Net(Module):   
    def __init__(self):
        super(Att_Net, self).__init__()

        self.cnn_layers = Sequential(
            # Defining a 2D convolution layer
            Conv2d(300, 256, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(256),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=2, stride=2),
            # Defining another 2D convolution layer
            Conv2d(256, 128, kernel_size=3, stride=1, padding=1),
            BatchNorm2d(128),
            ReLU(inplace=True),
            MaxPool2d(kernel_size=1, stride=1),
        )

        self.linear_layers = Sequential(
            Linear(128 * 20 * 1, 256),
            Linear(256, 64),
            Linear(64, 2),
        )

        self.attention = CBAM(gate_channels=128)

    # Defining the forward pass    
    def forward(self, x):
        x = self.cnn_layers(x)
        x = self.attention(x)
        x = x.view(x.size(0), -1)
        x = self.linear_layers(x)
        return x

In [18]:
# defining the model
model = Att_Net()
# defining the optimizer
optimizer = Adam(model.parameters(), lr=0.07)
# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    
print(model)

Att_Net(
  (cnn_layers): Sequential(
    (0): Conv2d(300, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(256, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=2560, out_features=256, bias=True)
    (1): Linear(in_features=256, out_features=64, bias=True)
    (2): Linear(in_features=64, out_features=2, bias=True)
  )
  (attention): CBAM(
    (ChannelGate): ChannelGate(
      (mlp): Sequential(
        (0): Flatten()
        (1): Linear(in_features=128, out_features=8, bias=True)
        (2): ReLU(

In [19]:
from torchsummary import summary
summary(model, (300, 40, 3))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 256, 40, 3]         691,456
       BatchNorm2d-2           [-1, 256, 40, 3]             512
              ReLU-3           [-1, 256, 40, 3]               0
         MaxPool2d-4           [-1, 256, 20, 1]               0
            Conv2d-5           [-1, 128, 20, 1]         295,040
       BatchNorm2d-6           [-1, 128, 20, 1]             256
              ReLU-7           [-1, 128, 20, 1]               0
         MaxPool2d-8           [-1, 128, 20, 1]               0
           Flatten-9                  [-1, 128]               0
           Linear-10                    [-1, 8]           1,032
             ReLU-11                    [-1, 8]               0
           Linear-12                  [-1, 128]           1,152
          Flatten-13                  [-1, 128]               0
           Linear-14                   

C:\Users\Asus\anaconda3\envs\tf-btp\lib\site-packages\torch\nn\functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [20]:
# # defining the number of epochs
# n_epochs = 1000
# # empty list to store training losses
# train_losses = []
# # empty list to store validation losses
# val_losses = []
# # training the model
# for epoch in range(n_epochs):
#     train(epoch)

In [21]:
# # prediction for training set
# with torch.no_grad():
#     output = model(train_x.cuda())
    
# softmax = torch.exp(output).cpu()
# prob = list(softmax.numpy())
# predictions = np.argmax(prob, axis=1)

# # accuracy on training set
# accuracy_score(train_y, predictions)

In [22]:
# import torch
# from torch.cuda.amp import GradScaler, autocast

# scaler = GradScaler()
# batch_size = 4
# gradient_accumulations = 16
# # this means training will be done for affective batch size of 4 * 16 = 64
# epochs = range(100)
# model.zero_grad()
# for epoch in epochs:
#         # Create a Batch
#         for batch_idx, batch in enumerate(zip(train_x,train_y)):
#             x, y = batch
#             with autocast():
#                 y_hat = model(x)
#                 loss = objective_function(y_hat, y)

#             scaler.scale(loss / gradient_accumulations).backward()

#             if (batch_idx + 1) % gradient_accumulations == 0:
#                 scaler.step(optimizer)
#                 scaler.update()
#                 model.zero_grad()

In [23]:
from torch.cuda.amp import GradScaler, autocast
n_epochs = 3000 # or whatever
batch_size = 100 # or whatever

for epoch in range(n_epochs):

    # X is a torch Variable
    permutation = torch.randperm(train_x.size()[0])

    for i in range(0,train_x.size()[0], batch_size):
        optimizer.zero_grad()

        indices = permutation[i:i+batch_size]
        batch_x, batch_y = train_x[indices], train_y[indices]

#         # in case you wanted a semi-full example
#         outputs = model.forward(batch_x)
#         loss = lossfunction(outputs,batch_y)
        with autocast():
                outputs = model(batch_x.cuda())
                loss = criterion(outputs,batch_y.to(torch.int64).cuda())
        
        
        loss.backward()
        optimizer.step()
        
    if epoch%50 == 0:
        print('Epoch : ',epoch+1, '\t', 'loss :', loss)

#         with torch.no_grad():
#             outputs = model(batch_x.cuda())
#         softmax = torch.exp(outputs).cpu()
#         prob = list(softmax.numpy())
#         predictions = np.argmax(prob, axis=1)

#         # accuracy on batch


#         print('Accuracy : ',epoch+1, '\t', 'accuracy :', accuracy_score(batch_y, predictions))
            

Epoch :  1 	 loss : tensor(6.2333, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  51 	 loss : tensor(0.6953, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  101 	 loss : tensor(0.6845, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  151 	 loss : tensor(0.6819, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  201 	 loss : tensor(0.6489, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  251 	 loss : tensor(0.3252, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  301 	 loss : tensor(0.0746, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  351 	 loss : tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  401 	 loss : tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  451 	 loss : tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  501 	 loss : tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  551 	 loss : tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>)
Epoch :  601 	 loss : tensor(nan, 

In [24]:
loss

tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>)

In [25]:
# prediction for training set
with torch.no_grad():
    output = model(train_x.cuda())
    
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on training set
accuracy_score(train_y, predictions)

0.4859184531315679

In [26]:
train_y

tensor([1., 0., 0.,  ..., 1., 0., 1.], dtype=torch.float64)

In [27]:
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)